# pytorchの特徴
- テンソル
- 自動微分
- DatasetとDataLoader
- GPUの利用

## テンソル

### 1 ライブラリとデータ作成

In [1]:
import numpy as np
import torch
torch.manual_seed(1)

#### 1.1 次元

In [2]:
x0 = torch.tensor([1.0, 2.0, 3.0])
print(x0.shape, x0)

torch.Size([3]) tensor([1., 2., 3.])


In [3]:
x3 = torch.tensor([[[1, 2, 3], [4, 5, 6], [7, 8, 9]], [[10, 11, 12], [13, 14, 15], [16, 17, 18]]])
print(x3.shape, x3, sep="\n")

torch.Size([2, 3, 3])
tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]]])


#### 1.2 連続データ

In [4]:
x = torch.arange(0, 10)
x

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

#### 1.3 ランダムデータ

In [5]:
x_3 = torch.rand(3, 3)
x_3

tensor([[0.7576, 0.2793, 0.4031],
        [0.7347, 0.0293, 0.7999],
        [0.3971, 0.7544, 0.5695]])

### 2 データの変形

- `numpy`では`reshape`だが, `pytorch`では`view`を使う
- `unsqueeze` 次元の追加

In [6]:
y = x_3.view(1, 9)
print(y.shape, y, sep="\n")

torch.Size([1, 9])
tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293, 0.7999, 0.3971, 0.7544, 0.5695]])


In [7]:
y = x_3.unsqueeze(1)
print(y.shape, y, sep="\n")

torch.Size([3, 1, 3])
tensor([[[0.7576, 0.2793, 0.4031]],

        [[0.7347, 0.0293, 0.7999]],

        [[0.3971, 0.7544, 0.5695]]])


### 3　データの参照

In [8]:
x = torch.randn(3,4)
x

tensor([[-0.6617, -0.0426, -1.3328,  0.5161],
        [ 0.7455, -0.0751, -0.6919, -0.4043],
        [ 0.2222,  0.5773, -1.7637,  0.2264]])

In [9]:
x[0,]

tensor([-0.6617, -0.0426, -1.3328,  0.5161])

In [10]:
x[:,2]

tensor([-1.3328, -0.6919, -1.7637])

### numpyとtensor

In [11]:
x = np.arange(0, 10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [12]:
y = torch.tensor(x)
y

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [13]:
y = torch.from_numpy(x)
y

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [14]:
y.numpy()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## 自動微分

### 1 ライブラリ

In [15]:
import torch
from torch import nn
torch.manual_seed(1)

### 2 線形変換と自動微分

In [17]:
x = torch.tensor(10)
w = torch.tensor(3, requires_grad=True, dtype=torch.float64)
b = torch.tensor(5, requires_grad=True, dtype=torch.float64)

In [18]:
def linear_model(x, w, b):
    return w * x + b

In [19]:
y = linear_model(x, w, b)
y

tensor(35., dtype=torch.float64, grad_fn=<AddBackward0>)

In [20]:
y.backward()

In [21]:
w.grad, b.grad

(tensor(10., dtype=torch.float64), tensor(1., dtype=torch.float64))

### 3 線形変換とMSE

In [22]:
x = torch.tensor(10)
w = torch.tensor(3, requires_grad=True, dtype=torch.float64)
b = torch.tensor(5, requires_grad=True, dtype=torch.float64)
t = torch.tensor(32)

In [23]:
def MSE(y, t):
    return (y-t)**2

In [24]:
y = linear_model(x, w, b)
L = MSE(y, t)

In [25]:
y, L

(tensor(35., dtype=torch.float64, grad_fn=<AddBackward0>),
 tensor(9., dtype=torch.float64, grad_fn=<PowBackward0>))

In [26]:
L.backward()

In [27]:
w.grad, b.grad

(tensor(60., dtype=torch.float64), tensor(6., dtype=torch.float64))

### 4 ライブラリの利用

In [28]:
x = torch.tensor([1, 2, 3, 4, 5]).unsqueeze(1).float()
t = torch.tensor([5, 8, 11, 14, 17]).unsqueeze(1).float()

In [30]:
linear = nn.Linear(1, 1)
MSE = nn.MSELoss()

In [31]:
y = linear(x)
L = MSE(y, t)

In [32]:
L.backward()

In [33]:
linear.weight, linear.bias

(Parameter containing:
 tensor([[0.5153]], requires_grad=True),
 Parameter containing:
 tensor([-0.4414], requires_grad=True))

In [34]:
linear.weight.grad, linear.bias.grad

(tensor([[-69.3125]]), tensor([-19.7912]))

## パラメータ更新

### 1 ライブラリ

In [35]:
import torch
from torch import nn
import torch.optim as optim
torch.manual_seed(1)

### 2 自動微分

In [36]:
x = torch.tensor([1, 2, 3, 4, 5]).unsqueeze(1).float()
t = torch.tensor([5, 8, 11, 14, 17]).unsqueeze(1).float()

In [37]:
linear = nn.Linear(1, 1)
MES = nn.MSELoss()

In [38]:
y = linear(x)
L = MSE(y, t)
L.backward()

In [39]:
linear.weight, linear.bias

(Parameter containing:
 tensor([[0.5153]], requires_grad=True),
 Parameter containing:
 tensor([-0.4414], requires_grad=True))

In [40]:
linear.weight.grad, linear.bias.grad

(tensor([[-69.3125]]), tensor([-19.7912]))

### 3 パラメータの更新

In [41]:
linear = nn.Linear(1, 1)
MSE = nn.MSELoss()
optimizer = optim.SGD(linear.parameters(), lr=1e-3)

In [42]:
linear.weight, linear.bias

(Parameter containing:
 tensor([[-0.1939]], requires_grad=True),
 Parameter containing:
 tensor([0.4694], requires_grad=True))

In [43]:
y = linear(x)
L = MSE(y, t)
L.backward()
optimizer.step()

In [44]:
linear.weight, linear.bias

(Parameter containing:
 tensor([[-0.1144]], requires_grad=True),
 Parameter containing:
 tensor([0.4916], requires_grad=True))

### 4 パラメータの推定

In [45]:
x = torch.tensor([1, 2, 3, 4, 5]).unsqueeze(1).float()
t = torch.tensor([5, 8, 11, 14, 17]).unsqueeze(1).float()

In [46]:
linear = nn.Linear(1, 1)
MSE = nn.MSELoss()
optimizer = optim.SGD(linear.parameters(), lr=1e-2)

In [48]:
for epoch in range(1000):
    y = linear(x)
    L = MSE(y, t)
    L.backward()
    optimizer.step()
    optimizer.zero_grad()

In [49]:
linear.weight, linear.bias

(Parameter containing:
 tensor([[3.0027]], requires_grad=True),
 Parameter containing:
 tensor([1.9903], requires_grad=True))

## DatasetとDataLoaderの扱い

### 1 ライブラリとデータのダウンロード

In [50]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
torch.manual_seed(1)

In [52]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0),(1)), lambda x: x.view(-1)])

root = './data'
mnist_train = datasets.MNIST(root=root, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root=root, train=False, download=True, transform=transform)

train_dataloder = DataLoader(mnist_train, batch_size=100, shuffle=True)
test_dataloder = DataLoader(mnist_test, batch_size=100, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [53]:
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0, std=1)
               <function <lambda> at 0x12b9e98a0>
           )

In [55]:
train_dataloder

### 2 データの参照

In [56]:
x, t = next(iter(train_dataloder))
x.shape, t.shape

(torch.Size([100, 784]), torch.Size([100]))

In [57]:
for x, t in train_dataloder:
    print(x.shape, t.shape)

torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100, 784]) torch.Size([100])
torch.Size([100,

### 3 自作Datasetクラス

#### 3.1 簡単なDataset

In [60]:
class Dataset:
    def __init__(self):
        self.data = [1, 2, 3, 4, 5]
        self.label = [0, 0, 0, 1, 1]
    def __getitem__(self, index):
        return self.data[index], self.label[index]
    def __len__(self):
        return len(self.data)

In [61]:
dataset = Dataset()

In [62]:
dataset

In [63]:
print(dataset[0], dataset[1], dataset[2], dataset[3], dataset[4])
print(len(dataset))

(1, 0) (2, 0) (3, 0) (4, 1) (5, 1)
5


In [70]:
class Dataset2:
    def __init__(self, transform_data=None, transform_label=None):
        self.data = [1, 2, 3, 4, 5, 6]
        self.label = [0, 0, 0, 1, 1, 1]
        self.transform_data = transform_data
        self.transform_label = transform_label
    def __getitem__(self, index):
        x = self.data[index]
        t = self.label[index]
        if self.transform_data:
            x = self.transform_data(self.data[index])
        if self.transform_label:
            t = self.transform_label(self.label[index])
        return x, t
    def __len__(self):
        return len(self.data)

In [71]:
dataset2 = Dataset2()

In [73]:
print(dataset2[0], dataset2[1], dataset2[2], dataset2[3], dataset2[4], dataset2[5])
print(len(dataset2))

(1, 0) (2, 0) (3, 0) (4, 1) (5, 1) (6, 1)
6


In [74]:
transform = lambda x: x + 10
dataset3 = Dataset2(transform_data=transform)

In [75]:
print(dataset3[0], dataset3[1], dataset3[2], dataset3[3], dataset3[4], dataset3[5])
print(len(dataset3))

(11, 0) (12, 0) (13, 0) (14, 1) (15, 1) (16, 1)
6


#### 3.3 Dataloaderに適応

In [76]:
dataloader = DataLoader(dataset3, batch_size=2, shuffle=True)

In [77]:
dataloader

In [78]:
for x, t in dataloader:
    print(x, t)

tensor([14, 15]) tensor([1, 1])
tensor([11, 16]) tensor([0, 1])
tensor([12, 13]) tensor([0, 0])
